In [ ]:
import pandas as pd
import numpy as np
import folium
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib import cm
import base64
from io import BytesIO

#parameters
file = 'Population.csv'
power = 4
grid_resolution = 500

#load data
df = pd.read_csv(file)
lats = df['Latitude'].values
lons = df['Longitude'].values
values = df['Population Within 1 km Radius'].values

#define grid size
lat_min, lat_max = lats.min(), lats.max()
lon_min, lon_max = lons.min(), lons.max()

grid_lat, grid_lon = np.mgrid[lat_min:lat_max:complex(grid_resolution),
                              lon_min:lon_max:complex(grid_resolution)]
grid_points = np.vstack((grid_lat.ravel(), grid_lon.ravel())).T

#inverse distance weighting function
def idw(x, y, z, xi, yi, power=4):
    dist = np.sqrt((xi[:, None]-x[None,:])**2+(yi[:,None]-y[None,:])**2)
    dist[dist==0] = 1e-10  #no division by zero
    weights = 1/dist**power
    weights /= weights.sum(axis=1, keepdims=True)
    zi = np.dot(weights, z)
    return zi

#interpolate values
interpolated = idw(lons, lats, values, grid_lon.ravel(), grid_lat.ravel())
interpolated_grid = interpolated.reshape(grid_lat.shape)

#generate colormap image
fig, ax = plt.subplots(figsize=(6, 6))
cmap = cm.get_cmap('viridis')
norm = plt.Normalize(vmin=interpolated_grid.min(), vmax=interpolated_grid.max())
ax.axis('off')
ax.imshow(interpolated_grid, cmap=cmap, extent=(lon_min, lon_max, lat_min, lat_max), origin='lower')

#save image
buf = BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, transparent=True)
plt.close(fig)
b64 = base64.b64encode(buf.getvalue()).decode('utf-8')
image_url = f'data:image/png;base64,{b64}'

#create folium map
center = [(lat_min+lat_max)/2, (lon_min+lon_max)/2]
m = folium.Map(location=center, zoom_start=13, tiles='cartodbpositron')

#overlay image
img_bounds = [[lat_min, lon_min], [lat_max, lon_max]]
folium.raster_layers.ImageOverlay(image_url, bounds=img_bounds, opacity=0.6).add_to(m)

#add data points (if you want)
'''
for lat, lon, pop in zip(lats, lons, values):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=f"Pop: {pop}",
        color='red',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)
'''

#save map
m.save('interpolated_population_map.html')
print("Map saved to interpolated_population_map.html")

/tmp/ipython-input-402454538.py:44: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('viridis')


Map saved to interpolated_population_map.html
